# Experiment: _Combined text-/graph features vs. text-only features_

In [ ]:
from notebook_prelude import *

In [ ]:
experiments = [
    'experiment_text_plain',
    'experiment_text_tfidf',
    #'experiment_graphs_plain',
    #'experiment_content_vs_structure_vs_both_content_only',
    #'experiment_split_multi_words_with_splitter',
    #'experiment_split_multi_words_with_splitter_and_lemmatizer'
] + [
    'experiment_combined' + x
    for x in [
        '_same_label',
        '_with_splitted_words',
        '_with_splitted_words_and_lemma',
        '_infrequent_removed',
        '_linearized'
    ]
]
df = results_helper.get_experiments_by_names(experiments, fetch_predictions=True)
df.loc[df.filename.str.contains('graph_text'), 'graph__preprocessing'] = 'GraphToTextTransformer'

## Result table

In [ ]:
group_attr = ['dataset',  'graph__preprocessing', 'type']
attr = 'prediction_score_f1_macro'
grouped = df.groupby(group_attr)[attr].max().to_frame().unstack().unstack().dropna(how='all', axis=1)
grouped

In [ ]:
print(grouped.to_latex(float_format=lambda x: "%.3f" % x))

## Significance tests

... between the concept maps combined and text-only

In [ ]:
df_confidence = significance_test_utils.get_confidences(df[df.type != TYPE_COOCCURRENCE], performance_attr=attr, model_selection_attr='type', model_selection_vals=[TYPE_CONCEPT_MAP, 'text'], log_progress=log_progress_nb)

In [ ]:
df_confidence.drop(columns='diffs')

In [ ]:
grouped.merge(df_confidence[['confidence']], left_index=True, right_index=True)